<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/RAG_lessons/blob/main/Pre_Retrieval_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Hướng dẫn về các biện pháp nâng cao hiệu quả cho RAG (pre-retrieval)**

Đây là notebook cho các bạn để có thể tìm hiểu về cách nâng cao hiệu quả cho RAG pipeline, với LlamaIndex và open-source LLM như Llama 3. Phương pháp thực hiện ở đây được triển khai trước retrieval.

***Cài đặt các thư viện cần thiết:***

Trước tiên, việc cần làm đầu tiên là cài đặt các thư viện cần thiết cho RAG pipeline. Điều này chúng ta có thể thực hiện tương tự như notebook trước.

In [1]:
!pip install --upgrade huggingface_hub==0.23.5
!pip install --upgrade peft==0.12
!pip install llama-index bitsandbytes accelerate llama-index-llms-huggingface llama-index-embeddings-huggingface
!pip install --upgrade sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.23.5 which is incompatible.
transformers 4.47.1 requires huggingface-hub<1.0,>=0.24.0, but you have huggingface-hub 0.23.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 22.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.23.5
    Uninstalling huggingface-hub-0.23.5:
      Successfully uninstalled hu

In [2]:
!pip install pypdf

Sau khi các bạn có được HuggingFace tokens và trữ trong phần Secrets, các bạn cần tạo biến "hf_token" để giúp cho Colab truy cập được vào token và LLM chúng ta sử dụng.

In [3]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

Tiếp theo đó, chúng ta cần tải tài liệu PDF xuống từ trên mạng. Các bạn có thể kéo thả trực tiếp tài liệu thông qua Google Colab folders, hoặc dùng các thư viện như requests, urllib để truy cập vào tài liệu. Ở đây chúng ta sẽ dùng urllib, và tải Form 10-K (form khai báo của công ty trước Ủy ban Giao dịch Chứng khoán Mỹ) của Alphabet (công ty mẹ của Google).

In [4]:
import urllib.request

url = "https://abc.xyz/assets/43/22/5deefff4fbec54014ae97b340c22/34ac6dab5f586b2e6e008b99fe683e35.pdf"

urllib.request.urlretrieve(url, "Alphabet2023Form10-K.pdf")


('Alphabet2023Form10-K.pdf', <http.client.HTTPMessage at 0x7cc67d897490>)

Tiếp đó chúng ta sẽ nhập các module chính của LlamaIndex (bao gồm vector DB chính) vào pipeline, tương tự như mục 1. Điểm khác biệt chính của chúng ta là chúng ta sẽ thêm một mục (SentenceWindowNodeParser) nhằm hiệu chỉnh thành phần tài liệu mà LLM truy lục (sentence window retrieval). Mỗi node trong tài liệu sẽ bao gồm 1 câu, và 1 "cửa sổ" bao gồm các câu ngay trước và ngay sau câu gốc.

Các bạn có thể tìm hiểu thêm về khái niệm này tại đây: [Metadata Replacement + Node Sentence Window
](https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/MetadataReplacementDemo/)

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

from llama_index.core.node_parser import SentenceWindowNodeParser, SentenceSplitter

reader = SimpleDirectoryReader(input_files=["/content/Alphabet2023Form10-K.pdf"])
data = reader.load_data()

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

Tiếp theo đó, chúng ta sẽ tạo system prompt, tương tự như notebook ở topic trước để đảm bảo tất cả câu trả lời đều hoàn toàn bằng tiếng Việt.

In [6]:
from llama_index.core import PromptTemplate
system_prompt = "Bạn là một trợ lý AI đắc lực. Hãy trả lời các câu hỏi càng chính xác càng tốt. Lưu ý là bạn chỉ trả lời bằng tiếng Việt và không dùng ngôn ngữ nào khác."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

***Indexing và lưu trữ tài liệu:***

Sau khi nhập tài liệu vào database, việc tiếp theo sẽ là tạo embedding cho từng phần nhỏ một (chunk), trước khi nhập embedding vào vector DB (tương tự như mô hình RAG cơ bản). Điểm khác biệt duy nhất là chúng ta cần thêm một bước để phân tách text.

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base", trust_remote_code=True)
text_splitter = SentenceSplitter()


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Việc nhập tokenizer, LLM (Llama-3-8B-Instruct) sẽ tương tự với notebook trước về mô hình RAG cơ bản.

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=hf_token,
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [9]:
# generate_kwargs parameters dựa vào https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    context_window=8192,
    max_new_tokens=512,
    model_kwargs={
        "token": hf_token,
        "torch_dtype": torch.bfloat16,
        "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": False, #do_sample cần phải là False cho temperature = 0.
        "temperature": 0.05,
        "top_p": 0.3,

    },
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_kwargs={"token": hf_token},
    stopping_ids=stopping_ids,
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Settings của notebook này tương tự như với notebook trước, chỉ khác là chúng ta cần text_splitter.

In [10]:
from llama_index.core import Settings


Settings.embed_model = embed_model

Settings.chunk_size = 512
# Llama-3-8B-Instruct model
Settings.llm = llm

Settings.text_splitter = text_splitter

***Đặt câu hỏi cho LLM (querying):***

Bước cuối cùng là đặt câu hỏi cho LLM thông qua vector database, tương tự như với notebook trước.

Tuy nhiên, do chúng ta thêm một bước nữa về sentence window retrieval, chúng ta cần tìm xem các node gốc (cho "window" về câu từ tài liệu chúng ta dùng) nằm ở đâu. Đó là lúc text_splitter bắt đầu được sử dụng.

In [11]:
index = VectorStoreIndex.from_documents(data)

nodes = node_parser.get_nodes_from_documents(data)
base_nodes = text_splitter.get_nodes_from_documents(data)


index = VectorStoreIndex(nodes)

base_index = VectorStoreIndex(base_nodes)

Các câu sau khi tìm được sẽ được load vào query engine với *MetadataReplacementPostProcessor*, và ở bước này thì "cửa sổ" gồm câu gốc và các câu liền kề sẽ thay thế hết các câu gốc ở mục *nodes*.

***Lưu ý:*** Vì biện pháp gốc (sentence window retrieval) được thực hiện trước phần truy lục, và bước thay thế được thực hiện tiếp, nên chúng ta có thể coi phương pháp này là tổng hợp giữa pre- và post-retrieval.

In [12]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor


query_engine = index.as_query_engine(similarity_top_k=3, streaming=True,
                                     node_postprocessors=[MetadataReplacementPostProcessor(target_metadata_key="window")
    ],)



Câu hỏi và câu trả lời có thể được đặt tương tự như với notebook cho RAG cơ bản.

In [13]:
response = query_engine.query("Tóm tắt thách thức với Alphabet Inc. trong năm 2023. "
 " Nêu từng thách thức với gạch đầu dòng, và dẫn nguồn theo từng trang cho mỗi thách thức.")

In [14]:
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Thách thức với Alphabet Inc. trong năm 2023 bao gồm:

• Tạm dừng hoạt động và giảm chi phí: Alphabet Inc. dự kiến sẽ phải trả phí khoảng 0,5 tỷ USD trong quý đầu tiên của năm 2023 để giảm chi phí hoạt động và giảm số lượng văn phòng. (Trang 32)

• Đánh giá lại tuổi thọ của các thiết bị và mạng: Alphabet Inc. đã đánh giá lại tuổi thọ của các thiết bị và mạng, dẫn đến việc giảm khấu hao khoảng 3,4 tỷ USD trong năm tài chính 2023. (Trang 84)

• Cải thiện báo cáo tài chính: Alphabet Inc. sẽ cập nhật báo cáo tài chính để phản ánh vai trò quan trọng của AI trong hoạt động của công ty. (Trang 84)

Note: The above answer is based on the provided context information and may not reflect the actual challenges faced by Alphabet Inc. in 2023.


Chúng ta có thể kiểm tra xem "cửa sổ", và câu gốc được phát hiện bởi LLM, nằm ở đâu như là một bước kiểm chứng cho hiệu quả của mô hình.

In [15]:
# for node in response.source_nodes:
#     print(node.id_)
#     print(node.node.get_content())
#     print("reranking score: ", node.score)
#     print("retrieval score: ", node.node.metadata["retrieval_score"])
#     print("=====================================")

window = response.source_nodes[0].node.metadata["window"]
sentence = response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: As a result we expect to incur exit costs
relating to office space reductions of approximately $0.5 billion in the first quarter of 2023.  We may incur
additional charges in the future as we further evaluate our real estate needs.
 • In January 2023, we completed an assessment of the useful lives of our servers and network equipment,
resulting in a change in the estimated useful life of our servers and certain network equipment to six years,
which we expect to result in a reduction of depreciation of approximately $3.4 billion for the full fiscal year 2023
for assets in service as of December 31, 2022, recorded primarily in cost of revenues and R&D expenses.
 • As AI is critical to delivering our mission of bringing our breakthrough innovations into the real world, beginning
in January 2023, we will update our segment reporting relating to certain of Alphabet's AI activities.  DeepMind,
previously reported within Other Bets, will be reported as part of Alphabet's corporate cost